<a href="https://colab.research.google.com/github/steineggerlab/foldcomp/blob/master/foldcomp-py-examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Foldcomp Python Example Notebook

Foldcomp compresses protein structures with torsion angles effectively. It compresses the backbone atoms to 8 bytes and the side chain to additionally 4-5 byes per residue, thus an averaged-sized protein of 350 residues requires ~4.2kb.

In this notebook we will show you the basic usage of the Foldcomp Python API.

In [ ]:
# Installing foldcomp
%pip install -q "foldcomp==0.0.3"

In [ ]:
# Download example PDB file
!wget -q https://raw.githubusercontent.com/steineggerlab/foldcomp/master/test/test.pdb

## Roundtrip a PDB file through Foldcomp

In this example we first compress and then decompress a PDB file, afterwards we load the atoms of both files into BioPython to compute a RMSD.

In [ ]:
# load foldcomp module
import foldcomp

# read example PDB file
with open("test.pdb", "r") as f:
  original = f.read()

# Compress input with reset points every 25 residues
# Should give a RMSD ~0.07A. A reset point every 200 residues will give a RMSD ~0.2A
fcz = foldcomp.compress("test.pdb", original, anchor_residue_threshold=25)

# Decompress again
(name, pdb) = foldcomp.decompress(fcz)

# Save as a PDB file again
with open(name + "_new.pdb", "w") as f:
    f.write(pdb)

In [ ]:
# install biopython
%pip install -q biopython

In [ ]:
# compute the RMSD between the two versions of the structure
import Bio.PDB

parser = Bio.PDB.PDBParser(QUIET = True)
reference = parser.get_structure("original", "test.pdb")
roundtrip = parser.get_structure("foldcomp", "test.pdb_new.pdb")

ref_atoms = [residue['CA'] for chain in reference[0] for residue in chain]
fcz_atoms = [residue['CA'] for chain in roundtrip[0] for residue in chain]

superposition = Bio.PDB.Superimposer()
superposition.set_atoms(ref_atoms, fcz_atoms)
superposition.apply([])

print(superposition.rms)

# Dealing with Foldcomp Databases

Dealing with Databases of enormous size such as the AlphaFold Database can be very painful due to the large number of files contained in it. We offer the AlphaFold Database in our own database format for easier download and handling.

Due to the limited amout of disk space in Google Colab, we will only download the Swiss-Prot subset of the database.

In [ ]:
# use the built-in downloader
# You can just call 
#   foldcomp.setup("afdb_swissprot_v4")
# in a local python environment. Google Colab requires a async call to work correctly
await foldcomp.setup_async("afdb_swissprot_v4")

In [ ]:
with foldcomp.open("afdb_swissprot_v4") as db:
  # Iterate through database
  i = 0
  for (name, pdb) in db:
    # save entries as seperate pdb files
    with open(name + ".pdb", "w") as f:
      f.write(pdb)
    i += 1
    if i % 10 == 0:
      break

The database setup command downloaded a database file containing all compressed structures in a single file and a few accompanying files. The "afdb_swissprot_v4.lookup" contains all the AlphaFold database accession mapping to the Foldcomp database. By giving a list of accessions to the `uniprot_ids` parameter in `foldcomp.open`, you can iterate over a user defined subset of accessions.

In [ ]:
!head afdb_swissprot_v4.lookup

In [ ]:
ids = ["AF-Q53M11-F1-model_v4", "AF-Q8IYB0-F1-model_v4"]
with foldcomp.open("afdb_swissprot_v4", ids = ids) as db:
  for (name, pdb) in db:
    with open(name + ".pdb", "w") as f:
      f.write(pdb)

Using `foldcomp.get(data)`, you can get a dictionary containing the precomputed
features of the structure. Here is an example of drawing Ramachandran plots for
a subset of the structures.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np

plt.style.use(
    'https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle'
)

list_of_data_dicts = []

# load the database
db = foldcomp.open("test/example_db")
for (name, pdb) in db:
    list_of_data_dicts.append((name, foldcomp.get_data(pdb)))

N_PROTEINS = 4

def set_axis_for_ramachandran(ax):
    ax.set_xlim(-180, 180)
    ax.set_ylim(-180, 180)
    ax.set_aspect("equal")
    ax.set_xticks([-180, -90, 0, 90, 180])
    ax.set_yticks([-180, -90, 0, 90, 180])
    ax.set_xlabel(r"$\phi$")
    ax.set_ylabel(r"$\psi$")

# plot the ramachandran plots
fig, ax = plt.subplots(figsize=(5, 5))
set_axis_for_ramachandran(ax)

for i, (name, data) in enumerate(list_of_data_dicts[:N_PROTEINS]):
    ax.scatter(data["phi"], data["psi"], s=1, label=name)

ax.legend()

plt.tight_layout()
plt.savefig("ramachandran.png", dpi=300)